In [ ]:
#| default_exp chatbot_tab

In [ ]:
#| export
import ipywidgets as widgets
import traitlets
from ipywidgets import Textarea, Text, Layout, HBox, Stack, Layout
from traitlets import HasTraits
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
from jupyter_mentor.chatbot import ChatBot, ChatBotModel

In [ ]:
from jupyter_mentor.chatbot import ChatBotModel
chatbot_model = ChatBotModel()
chatbot_model.prompt('hello')

## Extend ChatBotModel below

'Hello! How can I assist you today?'

In [ ]:

class ChatBotModel(FileModel):

    def __init__(self, bot_template="You are playing the role of a tutor/educator", human_template="{input_text}"):
        super().__init__()
        self.human_template = human_template
        self.human_message_prompt = HumanMessagePromptTemplate.from_template(self.human_template)
        self.update_bot_template(bot_template)

    def update_bot_template(self, bot_template):
        self.bot_template = bot_template
        self.bot_message_prompt = SystemMessagePromptTemplate.from_template(self.bot_template)
        self.chat_prompt = ChatPromptTemplate.from_messages([self.bot_message_prompt, self.human_message_prompt])

    def update_human_template(self, human_template):
        self.human_template = human_template
        self.human_message_prompt = HumanMessagePromptTemplate.from_template(self.human_template)
        self.chat_prompt = ChatPromptTemplate.from_messages([self.bot_message_prompt, self.human_message_prompt])        

    def prompt(self, input_text):
        ret = self.llm.invoke(input_text)
        return ret.content
    
    def prompt_with_template(self, kwargs):
        ret = self.llm.invoke(self.chat_prompt.format_prompt(**kwargs))
        return ret.content

In [ ]:
# read openapi key and set model
with open('OPENAI_API_KEY', 'r') as file:
    openai_api_key = file.read().strip()

llm = ChatOpenAI(openai_api_key = openai_api_key, model_name="gpt-3.5-turbo")

In [ ]:
student_template_options = widgets.RadioButtons(
    options={'Open Prompt': '{input_text}',
             'Metaphor':"I'm having trouble understanding {input_text}. Explain it as a metaphor."},
    description='Prompt Options:',
)
student_template_options

In [ ]:
student_template_options.value

NameError: name 'student_template_options' is not defined

In [ ]:
#| export
class StudentChatBot(widgets.VBox):

    #user = traitlets.CUnicode()
    #response = traitlets.CUnicode()
    #step_by_step = traitlets.Bool()
    #metaphor = traitlets.Bool()
    #hints = traitlets.Bool()
    #guided_questions = traitlets.Bool() 
    
    def __init__(self, chatbot_model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat_bot = ChatBot(chatbot_model)

        self.suggestion_buttons = HBox()
        self.step_by_step = widgets.Button(
            description='Step-by-Step',
            button_style='info',
            tooltip='Description',
        )
        self.metaphor = widgets.Button(
            description='Metaphor',
            button_style='info',
            tooltip='Description',
        )
        self.hints = widgets.Button(
            description='Hints',
            button_style='info',
            tooltip='Description',
        )
        self.guided_questions = widgets.Button(
            description='AI Guided Questions',
            button_style='info',
            tooltip='Description', 
        )
        self.suggestion_buttons.children = (self.step_by_step, self.metaphor, self.hints, self.guided_questions)
        self.children = (self.chat_bot, self.suggestion_buttons)

In [ ]:
student_chatbot = StudentChatBot(chatbot_model)
student_chatbot

StudentChatBot(children=(ChatBot(children=(Textarea(value='', disabled=True, layout=Layout(height='400px', wid…

In [ ]:
#| export
class EducatorChatBot(widgets.VBox):

    #user = traitlets.CUnicode()
    #response = traitlets.CUnicode()
    #step_by_step = traitlets.Bool()
    #metaphor = traitlets.Bool()
    #hints = traitlets.Bool()
    #guided_questions = traitlets.Bool()  
    
    def __init__(self, chatbot_model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat_bot = ChatBot(chatbot_model)

        self.suggestion_buttons = HBox()
        self.exam_questions = widgets.Button(
            description='Exam Questions',
            button_style='info',
            tooltip='Description',
        )
        self.lesson_plan = widgets.Button(
            description='Lesson Plan',
            button_style='info',
            tooltip='Description',
        )
        self.suggestion_buttons.children = (self.exam_questions, self.lesson_plan )
        self.children = (self.chat_bot, self.suggestion_buttons)

In [ ]:
educator_chatbot = EducatorChatBot(chatbot_model)
educator_chatbot

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()